In [ ]:
import sys

sys.path.append('../..')

In [ ]:
import dill

loaded_data = None

with open('../simulations/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

keys = loaded_data.keys()
keys.sort()
print keys

In [ ]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 15*step

maturity = loaded_data["randomization"]["maturity"]

#Discount

In [ ]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [ ]:
udlyings = loaded_data["underlyings"]

bm = udlyings[0]
time_grid = bm.time

# Derivative

In [ ]:
derivatives_nb = 1

In [ ]:
from finance.products.european.assetswap import (
    SwapContract,
)

swap_delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(bm, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

# Exposure

In [ ]:
from risk.exposures import EuropeanQuantileBrownianExposure

quantile_im = 0.80

im_drift = 0.3
im_vol = bm.vols[0][0]

exposure = EuropeanQuantileBrownianExposure(swap, discount, im_drift, im_vol)

# Indexes stuffs

## All idiosyncratic ids

In [ ]:
copula = loaded_data["credit"]["copula"]

all_ids = [list(sub[0])[0] for sub in copula.subsets if len(sub[0]) == 1]
obligors_nb = len(all_ids)

print "Obligors number: %s\n"%obligors_nb
print "All ids (including those that have a zero weight): %s"%all_ids

## Ids of $B$ and $C$

In [ ]:
b_id = 113

bc_ids = loaded_data["credit"]["bc_ids"]
c_ids = bc_ids[:]
c_ids.remove(b_id)

c_ids.sort()

print "Bank id: %s\n"%b_id
print "Counterparties ids: %s"%c_ids

## Subsets indexes of the copula for $B$ and $C$

In [ ]:
armaggeddon_subset_index = 129

In [ ]:
bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

b_subsets_indexes = copula.get_indexes_including(b_id)
b_subsets_indexes.remove(armaggeddon_subset_index)

c_subsets_indexes = bc_subsets_indexes[:]
for idx in b_subsets_indexes:
    c_subsets_indexes.remove(idx)
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n"%bc_subsets_indexes
print "Bank subsets indexes: %s\n"%b_subsets_indexes
print "Counterparties subsets indexes: %s"%c_subsets_indexes

# Portfolio construction

In [ ]:
import numpy as np

p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [ ]:
from finance.portfolio import EquilibratedPortfolio

pos = EquilibratedPortfolio.generate_1_vs_all_positions(b_id, bc_ids, obligors_nb)
positions = np.array(pos*notional).reshape(pos.size, 1)

In [ ]:
port = EquilibratedPortfolio(positions, [swap], [exposure])

# VM and IM accounts

In [ ]:
from ccp.accounts import Accounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [ ]:
# Be carefull, here, as the default models will be 
# used for regulatory computations, we select only 
# the first 125 idiosyncratic models even if 
# we care only about 61 over 125.

default_proba_models = copula.models[0 : obligors_nb]

# Regulatory capital

In [ ]:
from risk.basel import RegulatoryCapital

recoveries = [0.4 for i in range(0, obligors_nb)]

regul_cap = RegulatoryCapital(vm_accounts, im_accounts, port, default_proba_models, recoveries, bank_index=b_id)

# Funding and capital parameters

In [ ]:
lambda_ = 0.
recov_funding = 1.
k_ = 0.1

#MC loop

In [ ]:
from utils import time_offseter

In [ ]:
N = loaded_data["N"]

In [ ]:
results_idio = {"cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "dva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "fva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_ccr": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids}}

results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva_ccr": {"sum": 0., "sum2": 0.},
                 "kva_cva": {"sum": 0., "sum2": 0.}}

In [ ]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [ ]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    zeta_delta = time_offseter(zeta+delta, time_grid, True)
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]    
    b_min_df_time = default_times[b_df_times_indexes].min()
    
    cva_, dva_, fva_, kccr_, kcva_ = [0. for _ in range(5)]
    
    for c_id in c_ids:
        cp_subsets_indexes = copula.get_indexes_including(c_id)
        c_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in cp_subsets_indexes]
        
        bc_df_times_indexes = np.append(b_df_times_indexes, c_df_times_indexes)
        bc_df_times_indexes = np.sort(np.unique(bc_df_times_indexes))
        
        bc_df_times = default_times[bc_df_times_indexes]
        
        tau = bc_df_times.min()
        bar_tau = min(tau, maturity)
        
        cva__, dva__, fva__, kccr__, kcva__ = [0. for _ in range(5)]
        del bc_df_times
        
        if bar_tau <= zeta:
            continue
            
        inv_pdf = 1./exp_distrib.pdf(zeta)
        discount_zelta = discount(zelta)
        discount_zelta_delta = discount(zelta_delta)
        
        # VM computations
        d_pl = [s.price(zeta) for s in [swap]]
        b_vm = port.compute_value(d_pl, from_=b_id, towards_=c_id)
        vm_accounts.put_amounts(b_id, b_vm)
        vm_accounts.put_amounts(c_id, -b_vm)

        # IM computations
        b_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, \
                                     from_=b_id, towards_=c_id)
        c_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, \
                                     from_=c_id, towards_=b_id)
        
        im_accounts.put_amounts(b_id, b_im)
        im_accounts.put_amounts(c_id, c_im)
        
        p_zeta = b_vm.sum()        
        d_pl_zeta_delta = [s.price(zeta_delta) for s in [swap]]
        p_zeta_delta = port.compute_value(d_pl_zeta_delta, from_=b_id, towards_=c_id).sum()        
        # We suppose here that \Delta_{\zeta} is null
        q_zeta_delta = p_zeta_delta
        
        c_min_df_time = default_times[c_df_times_indexes].min()
        
        for df_time_index in bc_df_times_indexes:
            subset_index = bc_subsets_indexes[df_time_index]
            gamma_zeta = copula.gamma(subset_index, zeta)
            
            ####################
            ##### CVA part #####
            ####################
            c_default_indic = df_time_index in c_df_times_indexes
            if not c_default_indic:
                c_default_indic = c_min_df_time <= zeta_delta
                
            if c_default_indic:
                b_collat = p_zeta + im_accounts.get_amount(b_id).sum()
                loss = np.maximum(q_zeta_delta-b_collat, 0.)
                cva__ += gamma_zeta*(1.-recoveries[c_id]*loss)
                
                
            ####################
            ##### DVA part #####
            ####################
            b_default_indic = df_time_index in b_df_times_indexes
            if not b_default_indic:
                b_default_indic = b_min_df_time <= zeta_delta
                
            if b_default_indic:
                c_collat = p_zeta - im_accounts.get_amount(c_id).sum()
                loss = -np.minimum(q_zeta_delta-c_collat, 0.)
                dva__ += gamma_zeta*(1.-recoveries[b_id]*loss)
                
                
            ####################
            ##### FVA part #####
            ####################
            b_minus_funded_collat = p_zeta + \
                                    im_accounts.get_amount(b_id).sum() - \
                                    im_accounts.get_amount(c_id).sum()

            b_gamma_zeta = copula.tot_gamma(zeta, subsets_indexes=b_subsets_indexes)
            
            bar_lambda_zeta = .5*(1.-recoveries[b_id])*b_gamma_zeta
            tilde_lambda_zeta = bar_lambda_zeta - (1.-recov_funding)*b_gamma_zeta
            c_zeta = 0.
            lambda_zeta = lambda_
            
            fva__ += (c_zeta*b_minus_funded_collat + \
                      tilde_lambda_zeta*np.maximum(p_zeta-b_minus_funded_collat, 0.) + \
                      lambda_zeta*np.minimum(p_zeta-b_minus_funded_collat, 0.))
            
            
            ######################
            ##### K_CCR part #####
            ######################
            kccr__ += k_*regul_cap.compute_kccr(c_id, zeta)
            
            
            ######################
            ##### K_CCR part #####
            ######################
            kcva__ += k_*regul_cap.compute_kcva(c_id, zeta)
            
        results_idio['cva'][c_id]["sum"] += (inv_pdf*discount_zeta_delta*cva__)
        results_idio['cva'][c_id]["sum2"] += (inv_pdf*discount_zeta_delta*cva__)**2
        
        results_idio['dva'][c_id]["sum"] += (inv_pdf*discount_zeta_delta*dva__)
        results_idio['dva'][c_id]["sum2"] += (inv_pdf*discount_zeta_delta*dva__)**2
        
        results_idio['fva'][c_id]["sum"] += (inv_pdf*discount_zelta*fva__)
        results_idio['fva'][c_id]["sum2"] += (inv_pdf*discount_zelta*fva__)**2
        
        results_idio['kva_ccr'][c_id]["sum"] += (inv_pdf*discount_zelta*kccr__)
        results_idio['kva_ccr'][c_id]["sum2"] += (inv_pdf*discount_zelta*kccr__)**2
        
        results_idio['kva_cva'][c_id]["sum"] += (inv_pdf*discount_zelta*kcva__)
        results_idio['kva_cva'][c_id]["sum2"] += (inv_pdf*discount_zelta*kcva__)**2
        
        cva_ += inv_pdf*discount_zeta_delta*cva__
        dva_ += inv_pdf*discount_zeta_delta*dva__
        fva_ += inv_pdf*discount_zelta*fva__
        kccr_ += inv_pdf*discount_zelta*kccr__
        kcva_ += inv_pdf*discount_zelta*kcva__
    
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva_ccr"]["sum"] += kccr_
    results_globs["kva_ccr"]["sum2"] += kccr_**2
    
    results_globs["kva_cva"]["sum"] += kcva_
    results_globs["kva_cva"]["sum2"] += kcva_**2
                    
toc = time.time()

#Results

In [ ]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], \
                                                         copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "Confidence level used for IM: %.2f"%quantile_im

In [ ]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the individual xVA:\n"
keys = results_idio.keys()
keys.sort()

for k in keys:    
    print "%s:"%k.upper()
    print "---------------------------"
    v = results_idio[k]
    for id_, res_ in v.iteritems():        
        mean__ = res_['sum']/N
        mod_var__ = (res_['sum2']/N - mean__**2) / (N-1.)
        half_inter = z_level*np.sqrt(mod_var__)
        print "Id %s: [%f, %f]"%(id_, mean__-half_inter, mean__+half_inter)
    print

In [ ]:
print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)    